In [ ]:
pip install -q git+https://github.com/huggingface/transformers.git

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import ViTForImageClassification
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ViTFeatureExtractor, ViTImageProcessor

from pathlib import Path as path
from PIL import Image
import requests
import csv

import numpy as np
import autobot

In [3]:
val_ds = autobot.ImageNetSplit.from_path(
    folder_path=path.joinpath(path.home(), 'datasets', 'ImageNet'),
    csv_file='LOC_val_solution.csv',
    w=384, h=384,
    use_cache=True
)
val_loader = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=0)

In [4]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda:0')
device = torch.device('cpu')

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-384')
# model.eval()
# model.to(device)
model_mini = list(model.children())[0]
model_mini.eval()
model_mini.to(device)
print(model_mini)

ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0): ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation()
      

In [5]:
custom = autobot.vit.VisionTransformer.from_hugging_face(
    patch_size=16,
    emb_dim=768, num_heads=12,
    num_layers=12, 
    num_classes=1000,
    img_size=(384, 384)
)
custom.eval()
custom.to(device)
print()

In [15]:
for batch in val_loader:
    images, labels = batch['image'], batch['label']
    output_model = model_mini(images.to(device))[0]
    print('-'*30)
    output_custom = custom(images.to(device))

    # print(output_model.shape, output_custom.shape)
    # print(output_model[0, 0, :100])
    # print(output_custom[0, 0, :100])
    print(
        (output_model[0] == output_custom[0]).all()
    )
    break

b tensor([ 0.0112,  0.0268, -0.0706,  0.0612,  0.3059,  0.0327, -0.0222,  0.0220,
         0.0351, -0.0051], grad_fn=<SliceBackward0>)
logits tensor([ 6.1336, -0.6336, -1.0572, -1.0813, -0.7540, -1.1023, -1.5119, -1.4348,
        -1.6813, -0.8223], grad_fn=<SliceBackward0>)
------------------------------
b tensor([ 0.0112,  0.0268, -0.0706,  0.0612,  0.3059,  0.0327, -0.0222,  0.0220,
         0.0351, -0.0051], grad_fn=<SliceBackward0>)
logits tensor([ 6.1336, -0.6336, -1.0572, -1.0813, -0.7540, -1.1023, -1.5119, -1.4348,
        -1.6813, -0.8223], grad_fn=<SliceBackward0>)
tensor(False)


: 